In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import matplotlib as mpl
import matplotlib.pylab as plt2

plt2.rcParams["font.family"] = "NanumGothicCoding"
plt2.rcParams["font.size"] = 12
plt2.rcParams["figure.figsize"] = (10, 4)
mpl.rcParams['axes.unicode_minus'] = False

In [3]:
#2020년 실적만 불러오기

from skinet.utils.athena_tools import *
conn = athena_connection()
df = load_from_athena(conn, "select * from default.tm_list_detail where dialdate like '2020%'")
df.head()

/usr/local/lib/python3.6/dist-packages/fastparquet/dataframe.py:5: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import CategoricalIndex, RangeIndex, Index, MultiIndex


...tmp_db name = tmp_20200703_09_52_37_b218c0d402b773d33197eace2fc10c11b40d5f5e
Start creating table
drop table if exists default.tmp_20200703_09_52_37_b218c0d402b773d33197eace2fc10c11b40d5f5e
<< table not exist

        create  table         default.tmp_20200703_09_52_37_b218c0d402b773d33197eace2fc10c11b40d5f5e
        with (format='Parquet', parquet_compression='GZIP')
        as
        select * from default.tm_list_detail where dialdate like '2020%'
....
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 16)
Elapsed Time = 30.02
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/0aede89f-6ba4-4f94-944d-28ad6e883984 /home/skinet/tmp/tmp_20200703_09_52_37_b218c0d402b773d33197eace2fc10c11b40d5f5e --recursive
Elapsed Time = 14.29
>> Loading data
Dropping tmp db
<< Delete s3://adt-sandbox/dsc/tables/0aede89f-6ba4-4f94-944d-28ad6e883984
Removing tmp_dir!
Elapsed Time for downloading and loading = 20.03
Total Elapsed 

,listseqid,listid,custid,jisa,car,dialdate,dialtime,userid,resultcd,tmresultcd,...,custname,telno1,telno2,telno3,fax,zipcode,zipaddr,email,allocchargerid,work_dt
0,26384055,123796,215005434,,,2020/01/16,12:50:26,CI07225,NF05,,...,*,*,*,*,,*,*,*,CI07225,20200527
1,26389015,123798,100046106,,,2020/01/16,13:06:42,CI07225,SS03,,...,*,*,*,*,,*,*,*,CI07225,20200527
2,26392101,123798,100074627,,,2020/01/16,14:04:00,MPC02645,NF03,,...,*,*,*,*,,*,*,*,MPC02645,20200527
3,26375485,123796,217046877,,,2020/01/16,14:05:26,MPC02645,SE04,,...,*,*,*,*,,*,*,*,MPC02645,20200527
4,26392093,123798,100074514,,,2020/01/16,14:05:45,MPC02645,NF03,,...,*,*,*,*,,*,*,*,MPC02645,20200527


In [4]:
# 통화성공여부, 최종유치여부값
df['final_success'] = np.where(df['resultcd'] == 'SS01', 1, 0)
df['call_success'] = np.where((df['resultcd'] == 'SE04')|(df['resultcd'] == "NF03" )|(df['resultcd'] == 'SE05'), 0, 1)

In [5]:
# 결과코드 매핑

df['resultcd'] = df['resultcd'].map({
                                     'NF03' : '실패-무응답',
                                     'NF05' : '성공-가망재통화',
                                     'SE01' : '성공-TM거부',
                                     'SE04' : 'TM-자체제외',
                                     'SE05' : '오류번호',
                                     'SS01' : '성공-유치완료',
                                     'SS02' : '성공-중도취소',
                                     'SS03' : '성공-거절',
                                     'SS05' : '성공-단순재통화',
                                     'SS06' : '성공-부재중'   
                                   })

In [6]:
df.head(10)

,listseqid,listid,custid,jisa,car,dialdate,dialtime,userid,resultcd,tmresultcd,...,telno2,telno3,fax,zipcode,zipaddr,email,allocchargerid,work_dt,final_success,call_success
0,26384055,123796,215005434,,,2020/01/16,12:50:26,CI07225,성공-가망재통화,,...,*,*,,*,*,*,CI07225,20200527,0,1
1,26389015,123798,100046106,,,2020/01/16,13:06:42,CI07225,성공-거절,,...,*,*,,*,*,*,CI07225,20200527,0,1
2,26392101,123798,100074627,,,2020/01/16,14:04:00,MPC02645,실패-무응답,,...,*,*,,*,*,*,MPC02645,20200527,0,0
3,26375485,123796,217046877,,,2020/01/16,14:05:26,MPC02645,TM-자체제외,,...,*,*,,*,*,*,MPC02645,20200527,0,0
4,26392093,123798,100074514,,,2020/01/16,14:05:45,MPC02645,실패-무응답,,...,*,*,,*,*,*,MPC02645,20200527,0,0
5,26375690,123796,218121465,,,2020/01/22,16:45:31,K1190445,실패-무응답,,...,*,*,,*,*,*,K1190445,20200527,0,0
6,26380053,123796,218112934,,,2020/01/22,16:48:01,K1190445,성공-단순재통화,,...,*,*,,*,*,*,K1190445,20200527,0,1
7,26383713,123796,218071320,,,2020/01/22,16:53:00,K1190445,실패-무응답,,...,*,*,,*,*,*,K1190445,20200527,0,0
8,26369436,123796,214067242,,,2020/01/22,16:54:24,K1190445,실패-무응답,,...,*,*,,*,*,*,K1190445,20200527,0,0
9,26385680,123798,100000507,,,2020/01/22,16:56:47,K1170336,성공-거절,,...,*,*,,*,*,*,K1170336,20200527,0,1


In [7]:
df.drop(['listseqid', 'jisa', 'car', 'userid'], axis = 1, inplace = True)

In [8]:
drop_col = df.loc[:, 'tmresultcd' : 'allocchargerid'].columns
df.drop(drop_col, axis = 1, inplace = True)

In [9]:
df.drop_duplicates(inplace = True)

In [10]:
df.shape

(14800613, 8)

In [15]:
def reduce_mem_usage(df):

    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
        if str(col_type)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
               df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                 df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                 df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                 df[col] = df[col].astype(np.int64)

            else: 
                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16) 
                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                      df[col] = df[col].astype(np.float32) 
                 else:
                      df[col] = df[col].astype(np.float64) #else: #df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2 
    print('Memory usage of dataframe is {:.2f} MB --> {:.2f} MB (Decreased by {:.1f}%)'.format(
    start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))

    return df

In [16]:
reduce_mem_usage(df)

Memory usage of dataframe is 1016.28 MB --> 762.21 MB (Decreased by 25.0%)


,listid,custid,dialdate,dialtime,resultcd,work_dt,final_success,call_success
0,123796,215005434,2020/01/16,12:50:26,성공-가망재통화,20200527,0,1
1,123798,100046106,2020/01/16,13:06:42,성공-거절,20200527,0,1
2,123798,100074627,2020/01/16,14:04:00,실패-무응답,20200527,0,0
3,123796,217046877,2020/01/16,14:05:26,TM-자체제외,20200527,0,0
4,123798,100074514,2020/01/16,14:05:45,실패-무응답,20200527,0,0
...,...,...,...,...,...,...,...,...
14800608,123886,216089422,2020/02/24,11:22:51,오류번호,20200317,0,0
14800609,123886,214074321,2020/02/24,11:22:39,성공-거절,20200317,0,1
14800610,123886,217047659,2020/02/24,11:23:55,실패-무응답,20200317,0,0
14800611,123886,219055280,2020/02/24,11:24:48,성공-거절,20200317,0,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14800613 entries, 0 to 14800612
Data columns (total 8 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   listid         int32 
 1   custid         object
 2   dialdate       object
 3   dialtime       object
 4   resultcd       object
 5   work_dt        object
 6   final_success  int8  
 7   call_success   int8  
dtypes: int32(1), int8(2), object(5)
memory usage: 762.2+ MB


In [ ]:
df_sorted = df.sort_values(['custid', 'listid', 'dialdate', 'dialtime'])

In [ ]:
df_sorted

In [ ]:
df_sorted = df_sorted[['custid', 'listid', 'dialdate', 'dialtime', 'resultcd', 'final_success',
       'call_success', 'dialmonth']]

In [ ]:
df_sorted.head(20)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 5))